# LIBRARY

In [15]:
import os
import sys
import glob
import cv2
import time

import pandas as pd
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.utils.model_zoo import load_url as load_state_dict_from_url
import torchvision
import torchvision.transforms as transforms

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import albumentations as A

# SETTINGS

In [13]:
import warnings
warnings.filterwarnings('ignore')

PATH = os.path.join('..', 'input', 'aptos2019-blindness-detection')
PATH = os.path.join('..', 'input')

if torch.cuda.is_available():
    print('Cuda is available. GPU MODE!')
    device = 'cuda'
else:
    print('Cuda is not available. CPU MODE!')
    device = 'cpu'

Cuda is not available. CPU MODE!


# READ

In [5]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))

In [6]:
def prepare_labels(y):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder

y, le = prepare_labels(train['diagnosis'].values)

# DATASET

In [7]:
class DRDataset(Dataset):
    
    def __init__(self, df, datatype='train', transform=None, y=None):
        
        self.df = df
        self.datatype = datatype
        self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in self.df['id_code'].values]
        
        if self.datatype == 'train':
            self.labels = y
        else:
            self.labels = np.zeros((df.shape[0], 5))
        
        self.transform = transform
        
    def __len__(self):
        
        return len(self.image_files_list)
    
    def __getitem__(self, idx):
        
        img_name = self.image_files_list[idx]
        img = Image.open(img_name)
        
#       TODO: need to some extra steps to transfer into PIL
#         img = cv2.imread(img_name)
#         img = self.scale_radius(img, scale)
#         img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), scale / 30), -4, 128)
#         img_ = np.zeros(img.shape)
#         cv2.circle(img_, (img.shape[1] // 2, img.shape[0] // 2), int(scale * 0.9), (1, 1, 1), -1, 8, 0)
#         img = img * img_ + 128 * (1 - img_)

        image = self.transform(img)
        
        img_name_short = img_name.split('.')[0]

        label = self.labels[idx]
        if self.datatype == 'test':
            return image, label, img_name
        else:
            return image, label
        
    def scale_radius(img, scale):
        
        x = img[img.shape[0] // 2, :, :].sum(axis = 1)
        r = (x > x.mean() / 10).sum() / 2
        s = scale * 1.0 / r
        
        return cv2.resize(img, (0, 0), fx = s, fy = s)

# TRANSFORMS

In [9]:
transforms_train_list = []
transforms_test_list = []

if config['transforms_library'] == 'pytorch':
    if config['resize']:
        transforms_train_list.append(transforms.Resize(size=(config['image_size'], config['image_size'])))
    if config['randomHorizontalFlip']:
        transforms_train_list.append(transforms.RandomHorizontalFlip())
    if config['randomRotation']:
        transforms_train_list.append(transforms.RandomRotation())
    if config['toTrnsor']:
        transforms_train_list.append(transforms.ToTensor())
    if config['normalize']:
        transforms_train_list.append(transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
    train_transforms = transforms.Compose(transforms_train_list)
else:
    if config['resize']:
        transdorms_train_list.append(A.Resize(size=(config['image_size'], config['image_size'])))
    if config['horizontalFlip']:
        transforms_train_list.append(A.HorizontalFlip())
    if config['rotate']:
        transforms_train_list.append(A.Rotate())
    if config;['randomBrightness']:
        transforms_train_list.append(A.RandomBrightness())
    if config['randomContrast']:
        transforms_train_list.append(A.RandomContrast())
    if config['randomBrightnessContrast']:
        transforms_train_list.append(A.RandomBrightnessContrast())
    if config['hueSaturationValue']:
        transforms_train_list.append(A.HueSaturationValue())
    if config['toTensor']:
        transforms_train_list.append(A.torch.ToTensor())
    if config['normalize']:
        transforms_train_list.append(A.normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
    train_transforms = albumentations.Compose(transforms_train_list)

    
train_transforms = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=360),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# DATALOADER

In [11]:
train_dataset = DRDataset(df=train, datatype='train', transform=train_transforms, y=y)
test_dataset = DRDataset(df=test, datatype='test', transform=test_transforms)

tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis, test_size=0.1)

train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))

batch_size = 32
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

# MODEL

In [21]:
class DRDetect(nn.Module):
    
    def __init__(self):
        
        super(DRDetect, self).__init__()
        self.model = torchvision.models.resnet101()
        self.model.load_state_dict(torch.load(os.path.join('..', 'input', 'fastai-pretrained-models', 'resnet101-5d3b4d8f.pth')))
        slef.model.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.model.fc = nn.Sequential(
            nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Dropout(p=0.25),
            nn.Linear(in_features=2048, out_features=2048, bias=True),
            nn.ReLU(),
            nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=2048, out_features=1, bias=True),
        )
    
#     def make_graph():

    def forward(self, x):
    
        x = self.model(x)
        return x

In [22]:
model = DRDetect().to(device)

# TRAIN SETTINGS

In [27]:
lr = 0.003
momentum = 0.99
factor = 0.5
patience = 2

n_epochs = 20

# LOSS & OPTIMIZER & SCHEDULER

In [26]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=patience)

# TRAIN & VALID & TEST

In [28]:
def train(model, criterion, optimizer, train_loader, device):
    model.train()
    
    running_loss = 0
    for _, (data, target) in enumerate(train_loader):
        if device == 'cuda':
            data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        running_loss += loss.data
        
        loss.backward()
        optimizer.step()
        
    return running_loss / len(train_loader)

In [29]:
def valid(model, criterion, optimizer, valid_loader, device):
    model.eval()
    
    running_loss = 0
    for _, (data, target) in enumerate(valid_loader):
        if device == 'cuda':
            data, target = data.cuda(), target.cuda()
        
        output = model(data)
        loss = criterion(output, target)
        running_loss += loss.data
                
    return running_loss / len(valid_loader)

In [30]:
def test(model, test_loader, device, sub):
    model.eval()
    
    for (data, _, name) in test_loader:
        if device == 'cuda':
            data = data.cuda()
            
        output = model(data)
        output = output.cpu().detach().numpy()
        
        for i, (e, n) in enumerate(list(zip(output, name))):
            sub.loc[sub['id_code'] == n.split('/')[-1].split('.')[0], 'diagnosis'] = le.inverse_transform([np.argmax(e)])
   
    return sub

# RUN

In [ ]:
best_val = 1000000000

for epoch in tqdm(n_epochs):
    train_loss = train(model, criterion, optimizer, train_loader, device)
    val_loss = valid(model, criterion, optimizer, valid_loader, device)
    
    print('epoch {:d}, loss: {:.4f} val_loss: {:.4f}'.format(epoch, loss, val_loss))
    
    if val_loss < best_val:
        print('val_loss improved from {:.5f} to {:.5f}!'.format(best_val, val_loss))
        best_val = val_loss            
        model_file = 'best.pth'.format(epoch, val_loss)
        torch.save(model.state_dict(), os.path.join(log_dir, model_file))

# SUBMISSION

In [ ]:
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))
sub = test(model, test_loader, device, sub)
sub.to_csv('submission.csv', index=False)

In [6]:
import requests
from oauth2client import client
from oauth2client.file import Storage

In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('../input/credential.json', scope)
gc = gspread.authorize(credentials)
wks = gc.open('APTOS').sheet1

In [14]:
wks.update_acell('A1', 'Hello World!')
print(wks.acell('A1'))

<Cell R1C1 'Hello World!'>
